In [23]:
import numpy as np
import time

def dist(A, B, GramMatrix):
    return (
        np.diag(np.matmul(
            np.matmul(A, GramMatrix), 
            A.T)).reshape(-1, 1) +
        np.diag(np.matmul(
            np.matmul(B.T, GramMatrix), 
            B)) - 
        2 * (np.matmul(
            np.matmul(B.T, GramMatrix), 
            A.T))) 


A = np.matrix([[-1, 0, 8],
                        [1, 0, 10],
                        [1, 2, 20],
                        [20, -1, 0.56],
                        [-1, 0, -10]], dtype=np.float64)

B = np.matrix([[-1, 0, 8],
                        [1, 0, 10],
                        [1, 2, 20],
                        [20, -1, 0.56],
                        [-1, 0, -10]], dtype=np.float64).T

GramMatrix = np.matrix([[1, 0, 0],
                        [0, 1, 0],
                        [0, 0, 1]], dtype=np.float64)
 


result_shape = (A.shape[0], B.shape[1])


result1 = np.zeros((A.shape[0], B.shape[1]))

start_time1 = time.time()

for i in range(result_shape[0]):
    for j in range(result_shape[1]):
        result1[i, j] = np.dot(np.dot(A[i] - B[:,j].T, GramMatrix), A[i].T - B[:,j]).item()

end_time1 = time.time()
elapsed_time1 = end_time1 - start_time1

print(result1, elapsed_time1)

start_time2 = time.time()
result2 = dist(A, B, GramMatrix)
end_time2 = time.time()
elapsed_time2 = end_time2 - start_time2


print(result2, elapsed_time2)




[[  0.       8.     152.     497.3536 324.    ]
 [  8.       0.     104.     451.1136 404.    ]
 [152.     104.       0.     747.9136 908.    ]
 [497.3536 451.1136 747.9136   0.     553.5136]
 [324.     404.     908.     553.5136   0.    ]] 0.00099945068359375
[[  0.       8.     152.     497.3536 324.    ]
 [  8.       0.     104.     451.1136 404.    ]
 [152.     104.       0.     747.9136 908.    ]
 [497.3536 451.1136 747.9136   0.     553.5136]
 [324.     404.     908.     553.5136   0.    ]] 0.0
